# Data test

In [1]:
from tradingstrategy.chain import ChainId
from tradeexecutor.strategy.reserve_currency import ReserveCurrency
from tradingstrategy.timebucket import TimeBucket
import datetime
from tradingstrategy.client import Client

from tradeexecutor.strategy.execution_context import ExecutionMode
from tradeexecutor.strategy.trading_strategy_universe import load_pair_data_for_single_exchange, TradingStrategyUniverse
from tradeexecutor.strategy.execution_context import ExecutionContext
from tradeexecutor.strategy.universe_model import UniverseOptions
from tradingstrategy.client import Client



# Time bucket for our candles
CANDLE_TIME_BUCKET = TimeBucket.m15

# Candle time granularity we use to trigger stop loss checks
STOP_LOSS_TIME_BUCKET = TimeBucket.m5

# Strategy keeps its cash in USDC
RESERVE_CURRENCY = ReserveCurrency.usdc

# Which trading pair we are backtesting on
# (Might be different from the live trading pair)
# https://tradingstrategy.ai/trading-view/polygon/quickswap/eth-usdc
TRADING_PAIR = (ChainId.polygon, "uniswap-v3", "WETH", "USDC", 0.0005)


START_AT = datetime.datetime(2022, 9, 1)

# Backtest range
END_AT = datetime.datetime(2023, 4, 1)

def create_single_pair_trading_universe(
        ts: datetime.datetime,
        client: Client,
        execution_context: ExecutionContext,
        universe_options: UniverseOptions,
) -> TradingStrategyUniverse:

    # Fetch backtesting datasets from the server
    dataset = load_pair_data_for_single_exchange(
        client,
        time_bucket=CANDLE_TIME_BUCKET,
        pair_tickers=[TRADING_PAIR],
        execution_context=execution_context,
        universe_options=universe_options,
        stop_loss_time_bucket=STOP_LOSS_TIME_BUCKET,
    )

    # Convert loaded data to a trading pair universe
    universe = TradingStrategyUniverse.create_single_pair_universe(
        dataset,
        pair=TRADING_PAIR,
    )

    return universe

client = Client.create_jupyter_client()

universe = create_single_pair_trading_universe(
    END_AT,
    client,

    ExecutionContext(mode=ExecutionMode.data_preload),
    UniverseOptions()
)
pair = universe.get_single_pair()

print(f"We loaded {universe.universe.candles.get_candle_count():,} candles for pair #{pair.internal_id} {pair.get_ticker()}")
print(f"Data range is {universe.universe.candles.get_timestamp_range()}")

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy
We loaded 30,068 candles for pair #2854973 WETH-USDC
Data range is (Timestamp('2021-12-21 19:00:00'), Timestamp('2023-05-16 13:00:00'))
